# Example API Usage

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib
import tempfile

import numpy as np

from ska_pydada import AsciiHeader, DadaFile

## Create a DADA file

The cells below show how to create a DADA file with some random data.

The steps to create a DADA file are:
* create a header
* set the data
* serialise/dump to an output file

### Create ASCII Header

A header for the DADA file format is a simple key value structure that has at least 4096 bytes. The size
of the ASCII header is defined by the `HDR_SIZE` key and in the PyDADA library one can override this value
at construction time or via setting the property later.

```python
header = AsciiHeader(header_size=16384)
```

or

```python
header = AsciiHeader()
header.header_size = 16384
```

in the either example above the serialised header will have exactly 16384 bytes.

A header can be created from a string or a byte array (this is what is used when the `DadaFile` loads a DADA file).  The following
will use the `header_txt` variable to load the header.

In [3]:
header_txt = """HDR_SIZE            16384
HDR_VERSION         1.0
NCHAN               432
NBIT                32
NDIM                2
NPOL                2
RESOLUTION          1327104
UTC_START           2017-08-01-15:53:29
"""

In [4]:
header = AsciiHeader.from_str(header_txt)

In [5]:
assert header.header_size == 16384
assert int(header["NCHAN"]) == 432
assert int(header["NBIT"]) == 32
assert int(header["NDIM"]) == 2
assert int(header["NPOL"]) == 2
assert header.resolution == 1327104
assert header["UTC_START"] == "2017-08-01-15:53:29"

There are also utility methods on the `AsciiHeader` to get a header record as an `int` or `float`.

In [6]:
nbit = header.get_int("NBIT")
assert nbit == 32

Values can be added to the header either using a Python `dict` item insert or using the `set_value`

In [7]:
header["SOURCE"] = "J1644-4559_R"
assert header.get_value("SOURCE") == "J1644-4559_R"

# or

header.set_value("DESCRIPTION", "Description")
assert header["DESCRIPTION"] == "Description"

### Generate some data

For this notebook, the data will be random complex data with 768 time bins, 432 frequency channels, and 2 polarisations.

In [8]:
data = np.random.rand(768, 432, 2 * 2).astype(np.float32).view(np.complex64)
data.shape

(768, 432, 2)

In [9]:
data

array([[[0.5587505 +0.28362972j, 0.78835464+0.34670997j],
        [0.25524828+0.5732303j , 0.05809594+0.33555388j],
        [0.10852729+0.84412605j, 0.8548525 +0.90290314j],
        ...,
        [0.17367326+0.5577185j , 0.44578975+0.84860355j],
        [0.8628599 +0.64292485j, 0.23684965+0.41426903j],
        [0.0802725 +0.40919098j, 0.9897898 +0.53841424j]],

       [[0.6213365 +0.26848847j, 0.88289624+0.19796257j],
        [0.01431446+0.46715188j, 0.8041325 +0.985374j  ],
        [0.68946725+0.58263236j, 0.07418146+0.10708725j],
        ...,
        [0.8893546 +0.30932868j, 0.9021204 +0.4334117j ],
        [0.6286629 +0.35722712j, 0.52157533+0.04029623j],
        [0.8017936 +0.35487995j, 0.77741843+0.3462837j ]],

       [[0.48518205+0.3885295j , 0.7379586 +0.7069617j ],
        [0.9648924 +0.11118869j, 0.9648575 +0.9810868j ],
        [0.9185715 +0.12900384j, 0.38496187+0.8211594j ],
        ...,
        [0.05169209+0.27964845j, 0.45961565+0.6039255j ],
        [0.22235546+0.836947j

An instance of a `DadaFile` can be created using the constructor that takes an optional `AsciiHeader` and an optional by array of data.

However, the following show how to create a `DadaFile` and then set the data afterwards.

In [10]:
dada_file = DadaFile(header=header)
dada_file.set_data(data)

Once an instance of a `DadaFile` has been created, it can be saved as a file. This can be then be read back later

In [11]:
assert dada_file.data_size == len(data.tobytes())
dada_file.data_size

5308416

In [12]:
tmpdir = tempfile.gettempdir()
outfile = pathlib.Path(tmpdir) / "example_dada_file.dada"

dada_file.dump(outfile)

In [13]:
%ls -lh $outfile

-rw-rw-r-- 1 wgauvin wgauvin 5.1M May 29 13:57 /tmp/example_dada_file.dada


In [14]:
!head -c4096 $outfile

HDR_SIZE            16384
HDR_VERSION         1.0
NCHAN               432
NBIT                32
NDIM                2
NPOL                2
RESOLUTION          1327104
UTC_START           2017-08-01-15:53:29
SOURCE              J1644-4559_R
DESCRIPTION         Description
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

## Loading and reading DADA files

While the above shows how to create DADA files that is normally used for testing and is not the main focus of the PyDADA library or `DadaFile` itself. The power of the `DadaFile` is that it can read DADA files that conform to the DADA spec of having a header that has `HDR_SIZE` value of at least 4096 bytes and that the binary data is afterwards. As it is a flexible file format the data may be packed in different ways and this API provides general data access methods to get the data.

The following cell will read the file generated above and print out the header.

In [15]:
dada_file2 = DadaFile.load_from_file(outfile)
print(dada_file2.header)

HDR_SIZE            16384
HDR_VERSION         1.0
NCHAN               432
NBIT                32
NDIM                2
NPOL                2
RESOLUTION          1327104
UTC_START           2017-08-01-15:53:29
SOURCE              J1644-4559_R
DESCRIPTION         Description



To get the data in time, frequency and polarisation structure, once can use the `as_time_freq_pol` help method. There is no need to know how the data is layed out provided the header records are correct.

In [16]:
tfp_data = dada_file2.as_time_freq_pol()
tfp_data

array([[[0.5587505 +0.28362972j, 0.78835464+0.34670997j],
        [0.25524828+0.5732303j , 0.05809594+0.33555388j],
        [0.10852729+0.84412605j, 0.8548525 +0.90290314j],
        ...,
        [0.17367326+0.5577185j , 0.44578975+0.84860355j],
        [0.8628599 +0.64292485j, 0.23684965+0.41426903j],
        [0.0802725 +0.40919098j, 0.9897898 +0.53841424j]],

       [[0.6213365 +0.26848847j, 0.88289624+0.19796257j],
        [0.01431446+0.46715188j, 0.8041325 +0.985374j  ],
        [0.68946725+0.58263236j, 0.07418146+0.10708725j],
        ...,
        [0.8893546 +0.30932868j, 0.9021204 +0.4334117j ],
        [0.6286629 +0.35722712j, 0.52157533+0.04029623j],
        [0.8017936 +0.35487995j, 0.77741843+0.3462837j ]],

       [[0.48518205+0.3885295j , 0.7379586 +0.7069617j ],
        [0.9648924 +0.11118869j, 0.9648575 +0.9810868j ],
        [0.9185715 +0.12900384j, 0.38496187+0.8211594j ],
        ...,
        [0.05169209+0.27964845j, 0.45961565+0.6039255j ],
        [0.22235546+0.836947j

In [17]:
np.testing.assert_allclose(tfp_data, data)

The TFP data can also be retrieved by using the `data_c64` method one should include the shape.

In [18]:
tfp_data2 = dada_file2.data_c64(shape=(-1, 432, 2))
tfp_data2.shape

(768, 432, 2)

In [19]:
np.testing.assert_allclose(tfp_data2, data)

The raw data can be retrieved from the file by using:

In [20]:
raw_data = dada_file2.raw_data
len(raw_data), raw_data[:20]

(5308416, b'F\n\x0f?\xea7\x91>\x9c\xd1I?\xf8\x83\xb1>\xe7\xaf\x82>')

### Large files

This notebook as uses small files but data recorded during a scan can result in large files.  Loading the whole file into memory is not efficient
and the `DadaFile` defaults to only loading data of around 4MB at a time, the amount of data loaded will be equal to a multiple of `RESOLUTION` value defined in the header (this does default to 1 byte if not set).

The below shows using the `load_next` method to get the next lot of data.

In [21]:
data = np.random.rand(768 * 2, 432, 2 * 2).astype(np.float32).view(np.complex64)
data.shape

dada_file.set_data(data)
dada_file.dump(outfile)

/tmp/example_dada_file.dada already exists, overwriting it


In [22]:
%ls -lh $outfile

-rw-rw-r-- 1 wgauvin wgauvin 11M May 29 13:57 /tmp/example_dada_file.dada


In [23]:
dada_file3 = DadaFile.load_from_file(outfile)
len(dada_file3.raw_data)

5308416

In [24]:
raw_data1 = dada_file3.raw_data
bytes_read = dada_file3.load_next()
assert bytes_read == 5308416
raw_data2 = dada_file3.raw_data

np.testing.assert_raises(AssertionError, np.testing.assert_array_equal, raw_data1, raw_data2)

bytes_read = dada_file3.load_next()
assert bytes_read == 0
raw_data3 = dada_file3.raw_data
np.testing.assert_array_equal(raw_data2, raw_data3)

In [25]:
raw_data1[:10], raw_data2[:10], raw_data3[:10]

(b'\xf5U.>\xfb\xd4y?\x85R',
 b'\x08\x1c\xc2=\xa9\xca\xf5>\x04k',
 b'\x08\x1c\xc2=\xa9\xca\xf5>\x04k')

From the above raw data we see that first call to `load_next` returns new data but the next call doesn't.

Methods like the `as_time_freq_pol` can still be used after a read but it is on the latest chuck of data.

In [26]:
tfp = dada_file3.as_time_freq_pol()
tfp

array([[[0.09478003+0.4800618j , 0.8336642 +0.3908263j ],
        [0.41607982+0.1449619j , 0.852284  +0.6936452j ],
        [0.23709697+0.3317774j , 0.44195893+0.49377364j],
        ...,
        [0.50994176+0.5305429j , 0.98569727+0.14503585j],
        [0.22838905+0.03921705j, 0.45754528+0.37540248j],
        [0.13210799+0.17776819j, 0.99795014+0.0908934j ]],

       [[0.29261604+0.27494225j, 0.24017796+0.9610111j ],
        [0.56438327+0.5076644j , 0.7643466 +0.309219j  ],
        [0.7420254 +0.16324607j, 0.6229505 +0.5833704j ],
        ...,
        [0.6622303 +0.4142275j , 0.80861336+0.7390351j ],
        [0.5175124 +0.9603148j , 0.24045724+0.30193454j],
        [0.7661278 +0.05512405j, 0.66721994+0.48345447j]],

       [[0.14897986+0.18749909j, 0.6438592 +0.8601145j ],
        [0.2370368 +0.8618291j , 0.91966563+0.49942377j],
        [0.9485979 +0.34337658j, 0.52671355+0.68785983j],
        ...,
        [0.15438424+0.66827744j, 0.17978089+0.63260126j],
        [0.03632338+0.9523750

Note that the length of the raw dada is just over 5MB of data (it is `4 * RESOLUTION`). However, the file is around 11MB in size. More data can be loaded and processed by using the `load_next`

### Loading all the data

If you really need to load all the data in one go, you can pass a chunk size of `< 0` (i.e. -1) when creating the file.

In [27]:
dada_file4 = DadaFile.load_from_file(outfile, chunk_size=-1)

dada_file4.data_size

10616832

In [28]:
data = dada_file4.as_time_freq_pol()
data.shape

(1536, 432, 2)

## Clean up

In [29]:
if outfile.exists():
    outfile.unlink()